In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
energy = pd.read_csv('energydata_complete.csv')
energy.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [6]:
energy.columns

Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [3]:
# Extract data for T2 and T6
energy_lin_reg_df = energy[['T2','T6']]
energy_lin_reg_df


,T2,T6
0,19.200000,7.026667
1,19.200000,6.833333
2,19.200000,6.560000
3,19.200000,6.433333
4,19.200000,6.366667
...,...,...
19730,25.890000,24.796667
19731,25.754000,24.196667
19732,25.628571,23.626667
19733,25.414000,22.433333


In [10]:
# Firstly, we need to normalise our data set to a common scale using the min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(energy_lin_reg_df),columns=energy_lin_reg_df.columns)
features_df = normalised_df.drop(columns = ['T6'])
temp_target = normalised_df['T6']

In [14]:
# Now we split the dataset into training and and testing dataset
# Recall we have earlier segmented the festures and target variables

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
x_train, x_test, y_train, y_test = train_test_split(features_df,temp_target, test_size=0.3, random_state=1)

linear_model = LinearRegression()
# fit the model to the training dataset
linear_model.fit(x_train, y_train)
# obtain predictions
predicted_values = linear_model.predict(x_test)

#### Question 12: R^2 Value to 2 dp

In [13]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score,2)

0.65

In [16]:
new_energy = energy.drop(columns=['date','lights'])

#### Question 13: Mean Absolute Error

In [19]:
# Firstly, we need to normalise our data set to a common scale using the min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(new_energy),columns=new_energy.columns)
features_df = normalised_df.drop(columns = ['Appliances'])
temp_target = normalised_df['Appliances']

In [20]:
# Now we split the dataset into training and and testing dataset
# Recall we have earlier segmented the festures and target variables

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
x_train, x_test, y_train, y_test = train_test_split(features_df,temp_target, test_size=0.3, random_state=42)

linear_model = LinearRegression()
# fit the model to the training dataset
linear_model.fit(x_train, y_train)
# obtain predictions
predicted_values = linear_model.predict(x_test)

# Mean Absolute Error

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae,2)

0.05

#### Question 14: Residual Sum of Squares

In [21]:
### Residual Sum of Squares (RSS)
import numpy as np

rss = np.sum(np.square(y_test - predicted_values))
round(rss,2)

45.35

#### Question 15: Root Mean Squared Error

In [23]:
### Root Mean Square Error (RMSE)

from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test,predicted_values))
round(rmse,3)

0.088

#### Question 16: Co-efficient of Determination

In [24]:
# Coefficient of Determination - R Squared

from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score,2)

0.15

#### Question 18: Ridge Regression Model - RMSE Unchanged

In [30]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)
predicted_values = linear_model.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test,predicted_values))
round(rmse,3)

0.088

#### Question 20: RMSE with lasso regression

In [43]:
x_train, x_test, y_train, y_test = train_test_split(features_df,temp_target, test_size=0.3, random_state=42)
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)
predicted_values = linear_model.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test,predicted_values))
round(rmse,3)


0.088